In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt

In [11]:
from pathlib import Path
import numpy as np
import pandas as pd
from neurodsp.filt import filter_signal
from ecephys.signal.csd import kcsd_npix

In [4]:
from ecephys.graham import paths, channel_groups
from ecephys.sglx_utils import load_timeseries
from ecephys.scoring import load_visbrain_hypnogram
from ecephys.plot import plot_timeseries_interactive

In [5]:
condition = "SR"
subject = "Doppio"

## Load the data

In [8]:
sr_chans = channel_groups.stratum_radiatum_csd[subject]
so_chans = channel_groups.stratum_oriens_100um[subject]
hpc_chans = channel_groups.hippocampus[subject]
bin_path = Path(paths.lfp_bin[condition][subject])

In [25]:
(time, sr_lfps, fs) = load_timeseries(bin_path, sr_chans, start_time=None, end_time=None)

nChan: 385, nFileSamp: 18000019


In [26]:
(time, so_lfps, fs) = load_timeseries(bin_path, so_chans, start_time=None, end_time=None)

nChan: 385, nFileSamp: 18000019


In [9]:
(time, hpc_lfps, fs) = load_timeseries(bin_path, hpc_chans, start_time=None, end_time=None)

nChan: 385, nFileSamp: 18000019


## Explore LFPs

In [13]:
plot_timeseries_interactive(time, sr_lfps, chan_labels=sr_chans)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

In [12]:
lfps = (sr_lfps.T - so_lfps.T).T

In [14]:
plot_timeseries_interactive(time, lfps, chan_labels=sr_chans)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

## Filter signal

In [31]:
f_range = (None, 35)
filtered_sr_lfps = filter_signal(sr_lfps.T, fs, 'lowpass', f_range)

In [32]:
plot_timeseries_interactive(time, filtered_sr_lfps.T, chan_labels=sr_chans)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

In [73]:
f_range = (2, 35)
filtered_sr_lfps = filter_signal(sr_lfps.T, fs, 'bandpass', f_range)

In [74]:
plot_timeseries_interactive(time, filtered_sr_lfps.T, chan_labels=sr_chans)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

## Detection type I: Thresholding LFP negativity by Z-score

In [94]:
from ecephys.signal.ripples import _threshold_by_zscore
from ripple_detection.core import get_envelope

filtered_lfps = -filtered_sr_lfps.T
filtered_lfps[filtered_lfps < 0] = 0
not_null = np.all(pd.notnull(filtered_lfps), axis=1)
filtered_lfps, _time = (filtered_lfps[not_null], time[not_null])
combined_filtered_lfps = np.sum(filtered_lfps, axis=1)

candidate_spw_times = _threshold_by_zscore(
    combined_filtered_lfps,
    _time,
    minimum_duration=0.005,
    detection_zscore_threshold=3,
    boundary_zscore_threshold=1,
)

index = pd.Index(np.arange(len(candidate_spw_times)) + 1, name="spw_number")
spw_times = pd.DataFrame(candidate_spw_times, columns=["start_time", "end_time"], index=index)

In [95]:
from ecephys.signal.ripples import compute_ripple_features

spw_times = compute_ripple_features(_time, filtered_lfps, spw_times, fs, 'Kay', smoothing_sigma=0.0)

In [96]:
spw_times

,start_time,end_time,duration,center_time,nadir_time,envelope_integral,envelope_peak,mean_rms,summed_rms,max_rms,mean_amplitude,max_amplitude
spw_number,,,,,,,,,,,,
1,47.021553,47.048353,0.0268,47.034953,47.021553,166371.545583,2652.641939,584.854823,7603.112701,742.045104,526.215469,715.590225
2,73.973126,74.013126,0.0400,73.993126,74.013126,269764.957808,3110.843671,636.294686,8271.830917,727.651448,639.632100,740.460225
3,87.201112,87.301112,0.1000,87.251112,87.301112,758746.360762,3490.477184,710.592883,9237.707485,875.846920,716.534301,834.688976
4,87.999512,88.027911,0.0284,88.013711,87.999512,200044.981153,3034.285481,649.571194,8444.425524,839.666150,701.843824,756.933179
5,90.935109,90.964709,0.0296,90.949909,90.935109,181114.458519,2608.839470,580.888174,7551.546256,684.333483,439.888779,523.817146
...,...,...,...,...,...,...,...,...,...,...,...,...
6631,7197.235162,7197.277562,0.0424,7197.256362,7197.235162,277196.371691,2831.149172,605.984285,7877.795705,770.190965,525.649443,720.915793
6632,7198.137562,7198.175562,0.0380,7198.156562,7198.175562,227876.350072,2609.146032,528.896318,6875.652135,911.672230,406.677231,789.213268
6633,7198.634761,7198.716761,0.0820,7198.675761,7198.634761,958930.913475,4978.687382,1065.828176,13855.766290,1297.673928,1220.742873,1406.807306


In [97]:
from ecephys.plot import plot_ripple

from ipywidgets import (
    fixed,
    interact,
)

_, ax = plt.subplots(3, 1, figsize=(18, 6))
_ = interact(
    plot_ripple,
    time=fixed(_time),
    lfps=fixed(sr_lfps[not_null]),
    filtered_lfps=fixed(-filtered_lfps),
    fs=fixed(fs),
    ripple_times=fixed(spw_times),
    window_length=(0.25, 2, 0.25),
    ripple_number=(1, len(spw_times), 1),
    ax=fixed(ax),
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=1.0, description='window_length', max=2.0, min=0.25, step=0.25), IntSl…

## Detection type II: CSD

In [16]:
hpc_csd, estm_x = kcsd_npix(hpc_lfps)

In [17]:
plot_timeseries_interactive(time, hpc_csd.T)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

In [68]:
from ripple_detection.core import gaussian_smooth

smoothed_hpc_csd = gaussian_smooth(hpc_csd.T, 0.001, fs)

In [69]:
plot_timeseries_interactive(time, smoothed_hpc_csd)

/home/gfindlay/Documents/ecephys/ecephys/plot/plot.py:307: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  _, ax = plt.subplots(figsize=figsize)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

In [70]:
f_range = (1, 35)
filtered_hpc_lfps = filter_signal(hpc_lfps.T, fs, 'bandpass', f_range)

In [71]:
filtered_hpc_csd, estm_x = kcsd_npix(filtered_hpc_lfps.T)

In [72]:
plot_timeseries_interactive(time, filtered_hpc_csd.T)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()